### Importing Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

%matplotlib inline

### Loading 2014, 2015, 2016

In [2]:
df_2014 = pd.read_csv("2014_Declined.csv")
df_2015 = pd.read_csv("2015_Declined.csv")
df_2016 = pd.read_csv("2016_Declined.csv")
df_2017 = pd.read_csv("Decline_2017_Wells.csv")

In [3]:
final = df_2014

### Considering Action Type = 1, 2, 3, 6, 7
#### Approved : Action Type 1,2,6
#### Declined : Action Type : 3, 7

In [4]:
df = df_2017
df_2017 = df = df[(df["Action_Type"] == 1) | (df["Action_Type"] == 2) | (df["Action_Type"] == 3)| (df["Action_Type"] == 6)|(df["Action_Type"] == 7)]



In [5]:
final = final.append(df_2015)
final = final.append(df_2016)
final = final.append(df_2017)

In [6]:
final.shape

(368855, 28)

### All data is appended in Final Dataframe

In [7]:
interested = ['Agency_Code', 'Loan_Type','Property_Type', 'Loan_Purpose', 'Occupancy', 'Amount', 'Action_Type','State_Code', 'County_Code', 'Income', 'Purchaser_Type ','Property_Location', 'USPS_Code', 'GDP', 'Real_State_Growth_%', 'Rate','PMT', 'IncRat', 'Unemployemnt', 'AmorTerm', 'BoCreditScor', 'LTV']
data = final[interested]


### Categorical data is hot one encoding
### Continous data is Normalising 

In [8]:
cat = ['Agency_Code', 'Loan_Type', 'Property_Type', 'Loan_Purpose','Occupancy','USPS_Code' , 'County_Code','BoCreditScor', 'LTV' ]

no_cat = ['Amount', 'Income', 'GDP', 'Real_State_Growth_%', 'Rate', 'PMT', 'IncRat', 'Unemployemnt']

In [9]:
pre = pd.get_dummies(data, columns=cat, drop_first=True)

In [10]:
pre.shape

(368855, 389)

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scale = pd.DataFrame(data = pre)
scale[no_cat] = scaler.fit_transform(pre[no_cat])

# Show an example of a record with scaling applied
display(scale.head())

Amount  Action_Type  State_Code    Income  Purchaser_Type   \
0  0.00051            3          39  0.008502                0   
1  0.00454            2          48  0.052110                0   
2  0.00289            1           4  0.010602                6   
3  0.00113            6          16  0.008502                3   
4  0.00047            3          39  0.005501                0   

  Property_Location       GDP  Real_State_Growth_%  Rate       PMT   ...     \
0              Ohio  0.209551             0.666667   1.0  0.000544   ...      
1             Texas  0.590777             0.739130   1.0  0.004834   ...      
2           Arizona  0.093722             0.601449   1.0  0.003077   ...      
3             Idaho  0.012638             0.644928   1.0  0.001204   ...      
4              Ohio  0.209551             0.666667   1.0  0.000501   ...      

   County_Code_770  County_Code_775  County_Code_790  County_Code_800  \
0                0                0                0                0   
1                0                0                0                0   
2                0                0                0                0   
3                0                0                0                0   
4                0                0                0                0   

   County_Code_810  County_Code_820  County_Code_830  County_Code_840  \
0                0                0                0                0   
1                0                0                0                0   
2                0                0                0                0   
3                0                0                0                0   
4                0                0                0                0   

   BoCreditScor_4  LTV_1.0  
0               1        0  
1               1        0  
2               1        0  
3               1        0  
4               0        0  

[5 rows x 389 columns]

In [12]:
final = scale

In [13]:
t = final[288000:]

In [14]:
scale["Action_Type"].value_counts()

3    157887
1    115358
6     48000
2     47508
7       102
Name: Action_Type, dtype: int64

In [15]:
y = []
a = 0
d = 0
for i in final["Action_Type"]:
    if i ==1 or i ==2 or i ==6:
        a +=1
        y.append(1)
        
    elif i ==3 or i==7:
        d +=1
        y.append(0)

In [16]:
del final["Action_Type"]

In [17]:
del final['Property_Location']

In [18]:
final.shape

(368855, 387)

#### Train Data  = 2014, 2015, 2016 (288000 Sample)
#### Test Data = 2017 (50000 Sample)

In [19]:
temp = final

In [20]:
x_train = temp[:288000][:]

In [21]:
x_test = temp[288000:][:]

In [22]:
y_train = y[:288000]
y_test  = y[288000:]

In [23]:
len(y_test)

80855

In [24]:
x_test.shape

(80855, 387)

### Machine Learning : Logistic Regression

##### Training

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

clf = LogisticRegression()
clf.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
x_test["AmorTerm"] = x_test["AmorTerm"].fillna(360.0)

##### Testing

In [27]:
pred = clf.predict(x_test)

#### Accuracy

In [28]:
accuracy_score(y_test, pred)

0.8535526559891163

#### Confusion Matrix

In [29]:
print(confusion_matrix(y_test, pred))

[[10622  3367]
 [ 8474 58392]]
